In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten , Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
train_dir = r""
test_dir = r""

In [ ]:
train_gen = ImageDataGenerator(rescale = 1./255).flow_from_directory(train_dir,target_size = (150,150),batch_size = 32,class_mode = 'binary', shuffle=False)
test_gen = ImageDataGenerator(rescale = 1./255).flow_from_directory( test_dir,target_size=(150,150),batch_size = 32,class_mode = 'binary', shuffle=False)

In [ ]:
class_names = list(train_gen.class_indices.keys())
print("classes : ",class_names)

In [ ]:
model = Sequential([
    Conv2D(32,(3,3),activation = 'relu',input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation = 'relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128,activation = 'relu'),
    Dense(1,activation = 'sigmoid')
])

In [ ]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [ ]:
history = model.fit(train_gen,validation_data = test_gen,epochs = 5,verbose = 1)


In [ ]:
x_test,y_test = next(test_gen)
preds = (model.predict(x_test[:5]) > 0.5).astype(int).flatten()

In [ ]:
plt.figure(figsize = (12,4))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(x_test[i])
    plt.title(f"p : {class_names[preds[i]]}\nA : {class_names[int(y_test[i])]}")
    plt.axis('off')

plt.show()

In [ ]:
#eval_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir, target_size=(150,150), batch_size=32, class_mode='binary', shuffle=False)

y_true = eval_gen.classes
y_pred = (model.predict(eval_gen) > 0.5).astype(int).ravel()

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(cm, display_labels=class_names).plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Pneumonia Detection")
plt.show()